### Goal: Generate slices of fmri scans, then use a classfier to train on newly created slices

In [26]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib
from pathlib import Path


# SLICE GENERATOR

In [ ]:
devices = tf.config.list_physical_devices()
# print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)

In [28]:
test_img_path = os.path.join('data','fmri','beijing')
savepath = os.path.join('outputs','slice-outputs')
fmri_files = Path("data/fmri/beijing").glob("*.nii.gz")
count = 0

def slice_gen(imagename,x):
    
    slice_data = test_img_data[:,:,20,x]

    plt.imshow(slice_data, cmap='gray')
    plt.axis('off')  # Turn off axis labels
    plt.savefig(savepath + "/"+ imagename, bbox_inches='tight', pad_inches=0)
    plt.close()

for files in fmri_files:
    imgname = 'output' + str(count) + '.png'
    test_img = nib.load(str(files))
    test_img_data = test_img.get_fdata()
    #for x in range (0, 3):
    slice_gen(imgname,count)
    count+=1

# CLASSIFIER

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "outputs/slice-outputs"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

output0.png
output1.png
output10.png
output100.png
output101.png
output102.png
output103.png
output104.png
output105.png
output106.png
output107.png
output108.png
output109.png
output11.png
output110.png
output111.png
output112.png
output113.png
output114.png
output115.png
output116.png
output117.png
output118.png
output119.png
output12.png
output120.png
output121.png
output122.png
output123.png
output124.png
output125.png
output126.png
output127.png
output128.png
output129.png
output13.png
output130.png
output131.png
output132.png
output133.png
output134.png
output135.png
output136.png
output137.png
output138.png
output139.png
output14.png
output140.png
output141.png
output142.png
output143.png
output144.png
output145.png
output146.png
output147.png
output148.png
output149.png
output15.png
output150.png
output151.png
output152.png
output153.png
output154.png
output155.png
output156.png
output157.png
output158.png
output159.png
output16.png
output160.png
output161.png
output162.png
out

In [10]:
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
import os
import cv2
from PIL import Image
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

%matplotlib inline
keras.backend.set_image_data_format('channels_last')

In [7]:
devices = tf.config.list_physical_devices()
# print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)

GPU details:  {'device_name': 'METAL'}


## Before we can begin opening the images for processing, images need to be grayscaled properly first.

In [14]:
from pathlib import Path

directory = os.path.join('outputs','slice-outputs')
image_list = []
slice_outputs = Path(directory).glob('*.png')

# Iterate through the image files
for slice in slice_outputs:
    
    # converting image data to grayscale
    img = cv2.imread(str(slice))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(str(slice), gray)

    # making images operable
    img = Image.open(slice)

    img_array = np.array(img)
    img_array_flat = img_array.flatten()
    image_list.append(img_array_flat)

# Create a dataframe from the list of flattened image arrays
df = pd.DataFrame(image_list)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,164564,164565,164566,164567,164568,164569,164570,164571,164572,164573
0,3,3,3,3,3,3,3,5,5,5,...,8,8,8,8,10,10,10,10,10,10
1,2,2,2,2,2,2,2,1,1,1,...,5,5,5,5,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,1,1,1,1,1,1
3,2,2,2,2,2,2,2,2,2,2,...,4,4,4,4,3,3,3,3,3,3
4,3,3,3,3,3,3,3,5,5,5,...,0,0,0,0,2,2,2,2,2,2


In [15]:
def load_data_kfold(k):
    # Assuming the DataFrame df already contains flattened image arrays
    X_train = df.values  # Get the image data from the DataFrame as numpy array
    
    # If your images are grayscale or RGB, you may need to reshape them back
    # For example, if your images are 75x75:
    # X_train = np.array([img.reshape(75, 75, 3) for img in X_train])  # for RGB images
    # Or if grayscale:
    # X_train = np.array([img.reshape(75, 75) for img in X_train])
    
    # y_train is your target variable (this needs to be defined if not in the DataFrame)
    # For example, if the labels (is_iceberg) are in another list or DataFrame column:
    # y_train = your_labels  # This should match the size of your data
    
    # Placeholder for example labels (binary classification 0 or 1)
    y_train = np.random.randint(2, size=len(X_train))  # Replace with actual labels
    
    # Create stratified K-Folds
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(X_train, y_train))
    
    return folds, X_train, y_train

k = 7
folds, X_train, y_train = load_data_kfold(k)

In [18]:
def get_model():
    
    x = Input((75, 75, 3))
    model = BatchNormalization(axis = 3)(x)
    model = Convolution2D(filters = 32, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model)
    model = MaxPooling2D()(model)
    
    model = BatchNormalization(axis = 3)(model)
    model = Convolution2D(filters = 64, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model)
    model = MaxPooling2D()(model)
    
    model = BatchNormalization(axis = 3)(model)
    model = Convolution2D(filters = 128, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model)
    model = MaxPooling2D()(model)
    
    model = BatchNormalization(axis = 3)(model)
    model = Convolution2D(filters = 64, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model) 
    model = GlobalAveragePooling2D()(model)
 
    model = Dense(1, activation = 'sigmoid')(model)
    
    model = Model(inputs = x, outputs = model)
    
    opt_adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(opt_adam, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [19]:
model = get_model()
model.summary()

/opt/anaconda3/envs/slice-demo-1/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 75, 75, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 75, 75, 3)      │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 75, 75, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 37, 37, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 37, 37, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 18, 18, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 18, 18, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 18, 18, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 9, 9, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 9, 9, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 168,013 (656.30 KB)

 Trainable params: 167,559 (654.53 KB)

 Non-trainable params: 454 (1.77 KB)

In [20]:
batch_size=64

gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

In [21]:
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    return [mcp_save, reduce_lr_loss]

In [22]:
# First, print the shape of the data to check the size of the flattened images
print(X_train.shape)

# Assuming the flattened images have a different size than 75x75x3, adjust the reshape accordingly
# For example, if the images are grayscale and the size is 75x75:
# Replace 75x75x3 with 75x75x1 if grayscale, or calculate the correct dimensions if otherwise
for j, (train_idx, val_idx) in enumerate(folds):
    print('\nFold ', j)

    # Check the size of the fold
    print(f"Size of X_train_cv before reshaping: {X_train[train_idx].size}")
    print(f"Expected size after reshaping: {468*468*3}")
    
    # Update the dimensions based on the actual shape determined from the output
    height, width, channels = 468, 468, 1  # Adjust based on actual dimensions from the output

    X_train_cv = X_train[train_idx].reshape(-1, height, width, channels)
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train[val_idx].reshape(-1, height, width, channels)
    y_valid_cv = y_train[val_idx]
    
    name_weights = "final_model_fold" + str(j) + "_weights.h5"
    callbacks = get_callbacks(name_weights=name_weights, patience_lr=10)
    
    generator = gen.flow(X_train_cv, y_train_cv, batch_size=batch_size)
    
    model = get_model()
    
    model.fit(
        generator,
        steps_per_epoch=len(X_train_cv) // batch_size,
        epochs=15,
        shuffle=True,
        verbose=1,
        validation_data=(X_valid_cv, y_valid_cv),
        callbacks=callbacks
    )
    
    print(model.evaluate(X_valid_cv, y_valid_cv))

(194, 164574)

Fold  0
Size of X_train_cv before reshaping: 27319284
Expected size after reshaping: 657072


ValueError: cannot reshape array of size 27319284 into shape (468,468,1)